In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../data/SP500_EGS_Score_avarage_per_year.csv', index_col=0, parse_dates=True,sep=',')

In [ ]:
df.head(20)

In [ ]:
# all lower case

df.columns = map(str.lower, df.columns)

# - to _

df.columns = df.columns.str.replace('-', '_')

In [ ]:
len(df.company_symbol.unique())


In [ ]:
import sys 
sys.path.append('../')

In [ ]:
from helpers.text_preprocessing.preprocess_text import preprocess_text

In [ ]:
import pandas as pd

# Load or create your DataFrame (replace 'your_data.csv' with the path to your data file)
df = pd.read_csv('../data/extracted_text_sustainability_reports.csv', index_col=0, parse_dates=True,sep=',')


In [ ]:
# Preprocess the text and get the preprocessed DataFrame
preprocessed_df = preprocess_text(df)


In [1]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import plotly 
from gensim import corpora
import mlflow


/Users/nikitawilms/Documents/ffm-ds-23-2/ESG-Score-Prediction-from-Sustainability-Reports/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys 
sys.path.append('../')

In [3]:
from models.LDA_optuna_tuning.tune_lda_optuna import train_lda, compute_coherence
from models.LDA_optuna_tuning.call_optuna_tune import preprocess_data, execute_optuna_study

In [4]:
df = pd.read_csv('../data/preprocessed_data_text_format.csv', index_col=0, parse_dates=True,sep=',')

/var/folders/t5/tjfw6fh92wvd7nqk0dkpf15h0000gn/T/ipykernel_19402/2642038775.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../data/preprocessed_data_text_format.csv', index_col=0, parse_dates=True,sep=',')


In [5]:
df = df.head(200)

In [6]:
# write the preprocessed data to csv
df.to_csv('../data/preprocessed_data_text_format_200.csv')

In [7]:
preprocessed_stats = {
    'Number of Missing Values': df['preprocessed_content'].isna().sum(),
    'Number of Unique Texts': df['preprocessed_content'].nunique(),
    'Sample Texts': df['preprocessed_content'].sample(5).tolist()
}

preprocessed_stats

{'Number of Missing Values': 0,
 'Number of Unique Texts': 200,
 'Sample Texts': ['lead sustainable message message ceo pleased publish inaugural mental public company ask consider factor maximization near term proﬁt return capital strategic operational decision shareholder representative clear create durable value require enterprise add ress societal outcome enhance diversity equality sion ensure resilient healthy planet environmental impact business take continue active measure pursuit goal business inception critical enable mechanism circular economy measure disclosing manage dimension diversity carbon emission present necessary step company path improve outcome proud business nature enhance habitability planet facilit eat use recycling million vehicle year reduce environmental impact new vehicle part accelerate access aﬀordable transportation attendant beneﬁts social educational conomic mobility world discuss great detail stimate ﬁscal year business operation genera metric ton comp

In [8]:
df

,ticker,year,content,preprocessed_content
filename,,,,
ASX_ATCO_2019.pdf,ATCO,2019,Atlas Copco\nAnnual report\n2019Atlas Copco \n...,annual report cover ceo section contain vision...
NYSE_ACC_2019.pdf,ACC,2019,"ACC Cares: 2019 Environmental, Social & Govern...",environmental environmental commitment student...
TSX_CSH_2020.pdf,CSH,2020,"Chartwell Retirement Residences/Environmental,...",datum report accurate note environmental socia...
NYSE_ALK_2021.pdf,ALK,2021,CARE REPORT 2021\nALASKA AIRLINES 2021 SUSTA...,alaska sustainability content ceo people make ...
LSE_GKP.L_2021.pdf,GKP.L,2021,Sustainability report 2021GKP is a responsible...,sustainability report responsible energy compa...
...,...,...,...,...
NASDAQ_BIRD_2021.pdf,BIRD,2021,Allbirds \n2021 \nFlight \nStatus\nAllbirdsOu...,allbird version sustainability report document...
NYSE_JPM_2019.pdf,JPM,2019,Environmental\nSocial &\nGovernance \nReport\n...,environmental world contend human economic imp...
NYSE_RSG_2018.pdf,RSG,2018,1 2018 Sustainability Report |Our Blue Plane...,work ceo goal table content report reﬂect year...


In [9]:
if mlflow.active_run():
    mlflow.end_run()
execute_optuna_study(df, n_trials=200)

[I 2023-09-25 18:59:18,507] A new study created in RDB with name: no-name-cdc71fc3-167d-4620-a4dc-e5b9de02d1a7
[I 2023-09-25 18:59:33,628] Trial 0 finished with value: 0.40159224268755384 and parameters: {'alpha': 0.32850733383549807, 'eta': 0.7691810623783506, 'num_topics': 28}. Best is trial 0 with value: 0.40159224268755384.
[I 2023-09-25 18:59:48,896] Trial 1 finished with value: 0.37453960402361536 and parameters: {'alpha': 0.6286289552340938, 'eta': 0.6888843369441503, 'num_topics': 39}. Best is trial 0 with value: 0.40159224268755384.
[I 2023-09-25 19:00:01,448] Trial 2 finished with value: 0.40799766363651785 and parameters: {'alpha': 0.09451012677322512, 'eta': 0.8466556623015673, 'num_topics': 13}. Best is trial 2 with value: 0.40799766363651785.
[I 2023-09-25 19:00:18,239] Trial 3 finished with value: 0.3298463420005529 and parameters: {'alpha': 0.8109292234971223, 'eta': 0.16456058877358237, 'num_topics': 50}. Best is trial 2 with value: 0.40799766363651785.
[I 2023-09-25 1